In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)          
        y = np.load(y_path)           
        for i in range(X.shape[0]):
            for c in range(6):
                X[i, :, c] = (X[i, :, c] - X[i, :, c].mean()) / (X[i, :, c].std() + 1e-8)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(np.argmax(y, axis=1), dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class CNNEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU(),

            nn.AdaptiveAvgPool1d(1)  
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)  
        x = self.net(x)
        return x.squeeze(-1)   

class FullModel(nn.Module):
    def __init__(self, encoder, num_classes=62):
        super().__init__()
        self.encoder = encoder
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        emb = self.encoder(x)
        logits = self.classifier(emb)
        return logits

class EarlyStopping:
    def __init__(self, patience=7):
        self.patience = patience
        self.best_acc = 0.0
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.counter = 0
            self.best_state = {
                k: v.detach().cpu().clone()
                for k, v in model.state_dict().items()
            }
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

def train_and_eval(model, train_loader, test_loader,
                   epochs=500, patience=10):

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.0005)
    early_stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            logits = model(x)
            loss = criterion(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                preds = model(x).argmax(1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        val_acc = correct / total
        print(f"Epoch {epoch+1:02d} | Val Acc: {val_acc:.4f}")

        if early_stopper.step(val_acc, model):
            print("Early stopping")
            break

    early_stopper.restore(model)
    return early_stopper.best_acc

BASE = "models/Data/Data/62_classes/UserDependenet"

accs = []

for split in range(1, 11):
    print(f"\n===== Split {split} =====")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128)

    model = FullModel(CNNEncoder(), num_classes=62).to(device)

    acc = train_and_eval(model, train_loader, test_loader)
    accs.append(acc)

    print(f"Best accuracy (split {split}): {acc:.4f}")

print("\nFINAL AVERAGE ACCURACY (CNN + CCE):", np.mean(accs))

Using device: cuda

===== Split 1 =====
Epoch 01 | Val Acc: 0.1548
Epoch 02 | Val Acc: 0.3508
Epoch 03 | Val Acc: 0.4766
Epoch 04 | Val Acc: 0.5113
Epoch 05 | Val Acc: 0.5653
Epoch 06 | Val Acc: 0.6129
Epoch 07 | Val Acc: 0.6097
Epoch 08 | Val Acc: 0.6266
Epoch 09 | Val Acc: 0.6145
Epoch 10 | Val Acc: 0.6613
Epoch 11 | Val Acc: 0.6653
Epoch 12 | Val Acc: 0.6653
Epoch 13 | Val Acc: 0.6871
Epoch 14 | Val Acc: 0.6984
Epoch 15 | Val Acc: 0.6952
Epoch 16 | Val Acc: 0.7016
Epoch 17 | Val Acc: 0.6960
Epoch 18 | Val Acc: 0.6911
Epoch 19 | Val Acc: 0.7097
Epoch 20 | Val Acc: 0.7008
Epoch 21 | Val Acc: 0.7226
Epoch 22 | Val Acc: 0.7210
Epoch 23 | Val Acc: 0.7113
Epoch 24 | Val Acc: 0.7242
Epoch 25 | Val Acc: 0.7161
Epoch 26 | Val Acc: 0.7266
Epoch 27 | Val Acc: 0.7153
Epoch 28 | Val Acc: 0.7363
Epoch 29 | Val Acc: 0.7226
Epoch 30 | Val Acc: 0.7194
Epoch 31 | Val Acc: 0.7355
Epoch 32 | Val Acc: 0.7419
Epoch 33 | Val Acc: 0.7331
Epoch 34 | Val Acc: 0.7210
Epoch 35 | Val Acc: 0.7363
Epoch 36 | Val 

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)
        y = np.load(y_path)

        # channel-wise normalization (same as Keras)
        for i in range(X.shape[0]):
            for c in range(6):
                X[i, :, c] = (X[i, :, c] - X[i, :, c].mean()) / (X[i, :, c].std() + 1e-8)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(np.argmax(y, axis=1), dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ---------------- CNN + BiLSTM Model ----------------
class CNN_BiLSTM(nn.Module):
    def __init__(self, num_classes=62):
        super().__init__()

        # CNN part (same filters as Keras)
        self.cnn = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU()
        )

        # BiLSTM (same idea as Keras)
        self.bilstm = nn.LSTM(
            input_size=256,
            hidden_size=256,
            batch_first=True,
            bidirectional=True
        )

        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(512, num_classes)  # 256*2

    def forward(self, x):
        # x: (B, T, C)
        x = x.permute(0, 2, 1)       # (B, C, T)
        x = self.cnn(x)              # (B, 256, T')

        x = x.permute(0, 2, 1)       # (B, T', 256)
        out, _ = self.bilstm(x)      # (B, T', 512)

        feat = out[:, -1, :]         # last timestep
        feat = self.dropout(feat)
        logits = self.classifier(feat)

        return logits

# ---------------- Early Stopping ----------------
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_acc = 0.0
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.counter = 0
            self.best_state = {
                k: v.detach().cpu().clone()
                for k, v in model.state_dict().items()
            }
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

# ---------------- Train & Evaluate ----------------
def train_and_eval(model, train_loader, test_loader,
                   epochs=500, patience=10):

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            logits = model(x)
            loss = criterion(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # evaluation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                preds = model(x).argmax(dim=1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:03d} | Val Acc: {acc:.4f}")

        if stopper.step(acc, model):
            print("Early stopping triggered")
            break

    stopper.restore(model)
    return stopper.best_acc

BASE = "models/Data/Data/62_classes/UserDependenet"
accs = []

for split in range(1,11):
    print(f"\n===== SPLIT {split} =====")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128)

    model = CNN_BiLSTM(num_classes=62).to(device)

    acc = train_and_eval(model, train_loader, test_loader)
    accs.append(acc)

    print(f"Best accuracy (split {split}): {acc:.4f}")

    del model
    torch.cuda.empty_cache()

print("\nFINAL AVERAGE ACCURACY (CNN + BiLSTM + CCE):", np.mean(accs))

Using device: cuda

===== SPLIT 1 =====
Epoch 001 | Val Acc: 0.0258
Epoch 002 | Val Acc: 0.0460
Epoch 003 | Val Acc: 0.1073
Epoch 004 | Val Acc: 0.1540
Epoch 005 | Val Acc: 0.2226
Epoch 006 | Val Acc: 0.2677
Epoch 007 | Val Acc: 0.3468
Epoch 008 | Val Acc: 0.4113
Epoch 009 | Val Acc: 0.4516
Epoch 010 | Val Acc: 0.5032
Epoch 011 | Val Acc: 0.5274
Epoch 012 | Val Acc: 0.5468
Epoch 013 | Val Acc: 0.5685
Epoch 014 | Val Acc: 0.5645
Epoch 015 | Val Acc: 0.5790
Epoch 016 | Val Acc: 0.6097
Epoch 017 | Val Acc: 0.6218
Epoch 018 | Val Acc: 0.6427
Epoch 019 | Val Acc: 0.6484
Epoch 020 | Val Acc: 0.6403
Epoch 021 | Val Acc: 0.6645
Epoch 022 | Val Acc: 0.6444
Epoch 023 | Val Acc: 0.6653
Epoch 024 | Val Acc: 0.6581
Epoch 025 | Val Acc: 0.6863
Epoch 026 | Val Acc: 0.6758
Epoch 027 | Val Acc: 0.6839
Epoch 028 | Val Acc: 0.6774
Epoch 029 | Val Acc: 0.6952
Epoch 030 | Val Acc: 0.7008
Epoch 031 | Val Acc: 0.7000
Epoch 032 | Val Acc: 0.7008
Epoch 033 | Val Acc: 0.7097
Epoch 034 | Val Acc: 0.7129
Epoch 03